In [1]:
import os
import numpy as np
import pandas as pd
import ants
from tqdm import tqdm

In [2]:
ants.__version__

'0.6.1'

In [3]:
def get_design_matrix(epi,events_fn):
    from nilearn.glm.first_level import make_first_level_design_matrix

    events = pd.read_csv(events_fn,delimiter='\t')

    t_r = np.round(epi.spacing[-1],2)
    nTR = epi.shape[-1]
    n_scans = nTR
    frame_times = (np.arange(n_scans) * t_r)
    X1 = make_first_level_design_matrix(frame_times,events,drift_model="polynomial",drift_order=3,hrf_model="SPM") 
    return X1

In [4]:
indir = '../Data/020-fmriprepped'
subs = [s for s in os.listdir(indir) if all((s.startswith('sub'),not s.endswith('.html')))]
subs.sort()
n_orig = len(subs)
n_orig

34

In [5]:
# Parameters for looping. 
s = 30 # index for subject (out of 14)
r = 1 # index for run (out of 4)
sub = subs[s]

In [6]:
ofdir_root = '../Data/StudyForrest/ensembles_last_CVAE'
analysis_name = 'refactored-ABCD-v4'
ofdir = os.path.join(ofdir_root,analysis_name)

In [25]:
conf_fn_format = os.path.join(indir,sub,'ses-baselineYear1Arm1','func',f'{sub}_ses-baselineYear1Arm1_task-nback_run-0{r}_desc-confounds_timeseries.tsv')
events_fn_temp = os.path.join(f'../Data/011-ABCD-events/{sub}_ses-baselineYear1Arm1_task-nback_run-0{r}_events.tsv')

In [8]:
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix

In [41]:
kind = 'compcor'
for s in tqdm(range(n_orig),desc='subject loop'):
    for r in tqdm([1,2],desc='run loop',leave=False):
        epi_fn = os.path.join(ofdir,f'{kind}_S{s}_R{r}.nii.gz')
        epi = ants.image_read(epi_fn)
        sepi = epi.smooth_image(sigma=6,sigma_in_physical_coordinates=True,FWHM=True)
        sepi.to_filename(os.path.join(ofdir,f's{kind}_S{s}_R{r}.nii.gz'))

subject loop:  97%|█████████▋| 33/34 [28:39<00:52, 52.11s/it]


ValueError: File ../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/compcor_S33_R1.nii.gz does not exist!

In [43]:
for s in tqdm(range(11,34)): 
    kind = 'compcor'
    design_matrices = []
    n_scans = []
    func_imgs = []
    for r in [1,2]:
        epi_fn = os.path.join(ofdir,f's{kind}_S{s}_R{r}.nii.gz')
        epi = ants.image_read(epi_fn)

        nTR = epi.shape[-1]
        t_r = round(epi.spacing[-1],2)
        
        design_matrices.append( get_design_matrix(epi,events_fn_temp.format(s=s,r=r)) )
        n_scans.append(nTR)
        func_imgs.append(epi_fn)

    brain_mask_fn = os.path.join(indir,f'{sub}/ses-baselineYear1Arm1/anat/{sub}_ses-baselineYear1Arm1_run-01_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz')

    first_level_model = FirstLevelModel(
        t_r=t_r,
        slice_time_ref=0.5,
        hrf_model='spm',
        signal_scaling=False,
        drift_model='cosine',
        mask_img=brain_mask_fn,
    )

    first_level_model = first_level_model.fit(func_imgs, design_matrices=design_matrices)

    contrast_matrix = np.array([1,1,-3,1,1,1,-3,1,0,0,0,0,0,0]) # Face contrast

    z_map = first_level_model.compute_contrast(contrast_matrix, output_type='z_score')
    z_map.to_filename(os.path.join(ofdir,f'GLM-{kind}-face-S{s}.nii.gz'))

  0%|          | 0/23 [00:00<?, ?it/s]<ipython-input-43-fc4007f1880f>:32: UserWarning: One contrast given, assuming it for all 2 runs
  z_map = first_level_model.compute_contrast(contrast_matrix, output_type='z_score')
/home/aglinska/anaconda3/lib/python3.8/site-packages/nilearn/glm/_utils.py:205: RuntimeWarning: divide by zero encountered in true_divide
  return np.where(X <= 0, 0, 1.0 / X)
  4%|▍         | 1/23 [00:50<18:32, 50.58s/it]<ipython-input-43-fc4007f1880f>:32: UserWarning: One contrast given, assuming it for all 2 runs
  z_map = first_level_model.compute_contrast(contrast_matrix, output_type='z_score')
/home/aglinska/anaconda3/lib/python3.8/site-packages/nilearn/glm/_utils.py:205: RuntimeWarning: divide by zero encountered in true_divide
  return np.where(X <= 0, 0, 1.0 / X)
  9%|▊         | 2/23 [01:41<17:42, 50.59s/it]<ipython-input-43-fc4007f1880f>:32: UserWarning: One contrast given, assuming it for all 2 runs
  z_map = first_level_model.compute_contrast(contrast_matrix

ValueError: File ../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/scompcor_S33_R1.nii.gz does not exist!

In [46]:
contrast_maps = [os.path.join(ofdir,file) for file in os.listdir(ofdir) if file.startswith(f'GLM-{kind}-face-S')]
contrast_maps.sort()
contrast_maps

['../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S11.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S12.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S13.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S14.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S15.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S16.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S17.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S18.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S19.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S20.nii.gz',
 '../Data/StudyForrest/ensembles_last_CVAE/refactored-ABCD-v4/GLM-compcor-face-S

In [47]:
from nilearn.image import load_img, concat_imgs, math_img
from nilearn.glm.second_level import SecondLevelModel

# Create a design matrix for a one-sample t-test (all ones)
n_subjects = len(contrast_maps)
design_matrix = pd.DataFrame([1] * n_subjects, columns=['intercept'])

# Initialize and fit the second-level model
second_level_model = SecondLevelModel()
second_level_model = second_level_model.fit(contrast_maps, design_matrix=design_matrix)

# Compute the group-level (one-sample t-test) contrast
z_map = second_level_model.compute_contrast(second_level_contrast='intercept', output_type='z_score')

# Save the group-level z map
z_map.to_filename(os.path.join(ofdir,f'GLM-{kind}-face-group[.nii.gz'))

In [57]:
atype = 'contrast_face'
kind = 'preproc'
files = [os.path.join(ofdir,file) for file in os.listdir(ofdir) if all((file.startswith(f'{atype}_'),file.endswith(f'_{kind}.nii.gz')))]
files.sort()

In [58]:
arr = np.array([ants.image_read(file).numpy() for file in files]).mean(axis=0)
im = ants.image_read(files[0]).new_image_like(arr)
im.to_filename(os.path.join(ofdir,f'{atype}_{kind}_grandavg.nii.gz'))

In [60]:
def sphere(shape, radius, position):
    """Generate an n-dimensional spherical mask."""
    # assume shape and position have the same length and contain ints
    # the units are pixels / voxels (px for short)
    # radius is a int or float in px
    assert len(position) == len(shape)
    n = len(shape)
    semisizes = (radius,) * len(shape)

    # genereate the grid for the support points
    # centered at the position indicated by position
    grid = [slice(-x0, dim - x0) for x0, dim in zip(position, shape)]
    position = np.ogrid[grid]
    # calculate the distance of all points from `position` center
    # scaled by the radius
    arr = np.zeros(shape, dtype=float)
    for x_i, semisize in zip(position, semisizes):
        # this can be generalized for exponent != 2
        # in which case `(x_i / semisize)`
        # would become `np.abs(x_i / semisize)`
        arr += (x_i / semisize) ** 2

    # the inner part of the sphere will have distance below or equal to 1
    return arr <= 1.0

In [ ]:
sphere(shape, radius, position)

In [64]:
temp = ants.image_read('../Misc/single_subj_T1.nii')

/home/aglinska/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [102]:
# temp.plot_ortho(im,flat=True,xyz=(69,38,26))
# im_sphere = im.new_image_like(sphere(im.shape,10,(69,38,26))*1.0)
# im_sphere.to_filename(os.path.join(ofdir,'sphere.nii'))
# print(im_sphere.numpy().sum())
# temp.plot_ortho(im_sphere,flat=True,xyz=(20,38,26))

/home/aglinska/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [115]:
sphere = ants.image_read(os.path.join(ofdir,'sphere.nii'))

In [170]:
res = np.zeros((2,33,3))
for s in range(33):
    for r in [1,2]:
        im_p = ants.image_read(os.path.join(ofdir,f'contrast_face_{s}_R{r}_preproc.nii.gz'))
        im_c = ants.image_read(os.path.join(ofdir,f'contrast_face_{s}_R{r}_compcor.nii.gz'))
        im_d = ants.image_read(os.path.join(ofdir,f'contrast_face_{s}_R{r}_deepcor.nii.gz'))

        critval = np.sort(im_c.numpy()[sphere.numpy()==1].flatten())[-501]
        indiv_roi = (im_c.numpy()>critval) * sphere.numpy()==1

        res[r-1,s,0] = im_p.numpy()[indiv_roi].mean()
        res[r-1,s,1] = im_c.numpy()[indiv_roi].mean()
        res[r-1,s,2] = im_d.numpy()[indiv_roi].mean()

In [171]:
from matplotlib import pyplot as plt
from scipy.stats import ttest_rel
res.mean(axis=0).mean(axis=0)

array([3.56413949, 3.95270779, 4.75079769])

In [172]:
ttest_rel(res.mean(axis=0)[:,1],res.mean(axis=0)[:,2])

Ttest_relResult(statistic=-3.4968440822165614, pvalue=0.0014046508296383933)